## Random model tutorial


In this notebook, we present a more verbose version of the standard submission.py script, with the aim of explaining in detail how the main abstractions work and showing how easy it is to partecipate in the challenge. 

_NOTE_: this notebook is meant as a coding guide to the evaluation script, and a walk-through baseline submission to explain how to partecipate in the challenge. While you're free to experiment with this or other notebooks and even submit to the leaderboard from here, the _final_ submission should comply with the template scripts, as explained in the README.

Please contact the organizers on Slack should you have any doubt.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
sys.path.insert(0, '../')

_Basic imports, read the credentials from the env file_

In [ ]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv

load_dotenv('../upload.env')

EMAIL = os.getenv('EMAIL')  # the e-mail you used to sign up
assert EMAIL != '' and EMAIL is not None
BUCKET_NAME = os.getenv('BUCKET_NAME') # you received it in your e-mail
PARTICIPANT_ID = os.getenv('PARTICIPANT_ID') # you received it in your e-mail
AWS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY') # you received it in your e-mail
AWS_SECRET_KEY = os.getenv('AWS_SECRET_KEY') # you received it in your e-mail
UPLOAD = bool(os.getenv('UPLOAD'))  # it's a boolean, True if you want to upload your submission
LIMIT = int(os.getenv('LIMIT'))  # limit the number of test cases, for quick tests / iterations. 0 for no limit

print("Submission will be uploaded: {}".format(UPLOAD))
if LIMIT > 0:
    print("WARNING: only {} test cases will be used".format(LIMIT))

_NOTE: as long as there is a limit specified, the runner won't upload results: make sure to have LIMIT=0 when you want to submit to the leaderboard!_

In [ ]:
from evaluation.EvalRSRunner import EvalRSRunner

_Declare our model, in this case, a random generator: any model needs to include an implementation of "predict", taking user IDs as input and returning a DataFrame with predictions as output._

In [ ]:
class RandomModel:
    def __init__(self, items: pd.DataFrame):
        self.items = items

    def predict(self, user_ids: pd.DataFrame, k=20) -> pd.DataFrame:
        user_ids = user_ids['user_id'].values
        num_users = len(user_ids)
        pred = self.items.sample(n=k*num_users, replace=True)['track_id'].values
        pred = pred.reshape(num_users, k )
        pred = np.concatenate((np.array(user_ids).reshape(-1,1), pred), axis=1)
        return pd.DataFrame(pred, columns=['user_id', *[ str(i) for i in range(k)]]).set_index('user_id')

_We inherit from EvalRSRunner, and implement the required method, train_model: train_model encapsulate all your training logic, and should return any model class, wrapping predictions as shown above._

RandomEvalRSRunner is the Python object that will run the evaluation loop and provide utility functions to access data assets, tests, and upload results to the leaderboard.

In [ ]:
class RandomEvalRSRunner(EvalRSRunner):
    def train_model(self, train_df: pd.DataFrame):
        """
        Implement here your training logic. Since our example method is a simple random model,
        we actually don't use any training data to build the model, but you should ;-)

        At the end of training, you should return a model class that implements the `predict` method,
        as RandomModel does.
        """
        return RandomModel(self.df_tracks)

_We initiliaze the runner with our credentials_

In [ ]:
runner = RandomEvalRSRunner(
    num_folds=4,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    participant_id=PARTICIPANT_ID,
    bucket_name=BUCKET_NAME,
    email=EMAIL)

_Let's inspect the main data assets first_

In [ ]:
runner.df_tracks.head()

In [ ]:
runner._get_train_set(3).head()

In [ ]:
runner.df_users.head()

_Finally, we run the evaluation code_

In [ ]:
runner.evaluate(upload=UPLOAD, limit=LIMIT)

### Final submission to the committee

Since this is a code competition, you'll be required to submit your repository for statistical tests. Please consult the README carefully to make sure your project complies with the rules and follows the provided template script.